<a href="https://colab.research.google.com/github/dattasumanta619-del/Cattle-Breed-Identifier/blob/main/MK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1.0 STARTUP: mount Drive and copy dataset+model into /content (persistent -> visible)
from google.colab import drive
import os, shutil

drive.mount('/content/drive', force_remount=False)

# ========== EDIT these paths if your Drive layout differs ==========
DRIVE_DATA_ROOT = "/content/drive/MyDrive/indian_bovine_dataset"   # should contain folder "Indian_bovine_breeds"
DRIVE_MODEL_DIR  = "/content/drive/MyDrive/buffalo_models"         # should contain best_model.pth
MODEL_FILENAME   = "best_model.pth"
# ==================================================================

LOCAL_DATA_ROOT = "/content/indian_bovine_dataset"
LOCAL_MODEL_PATH = os.path.join("/content", MODEL_FILENAME)

# Copy dataset to /content (only if not already copied)
if os.path.exists(DRIVE_DATA_ROOT) and not os.path.exists(LOCAL_DATA_ROOT):
    print("Copying dataset from Drive to /content ...")
    shutil.copytree(DRIVE_DATA_ROOT, LOCAL_DATA_ROOT)
    print("✅ Dataset copied to", LOCAL_DATA_ROOT)
else:
    print("Dataset already in /content or missing in Drive. Check path:", DRIVE_DATA_ROOT)

# Copy model to /content (so notebook user can load without re-training)
drive_model_path = os.path.join(DRIVE_MODEL_DIR, MODEL_FILENAME)
if os.path.exists(drive_model_path):
    shutil.copy(drive_model_path, LOCAL_MODEL_PATH)
    print("✅ Model copied to", LOCAL_MODEL_PATH)
else:
    print("⚠️ Model not found in Drive at:", drive_model_path)

# set global DATA_DIR used later
DATA_DIR = os.path.join(LOCAL_DATA_ROOT, "Indian_bovine_breeds")
print("DATA_DIR ->", DATA_DIR)
print("Local model ->", LOCAL_MODEL_PATH if 'LOCAL_MODEL_PATH' in globals() else "(none)")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copying dataset from Drive to /content ...
✅ Dataset copied to /content/indian_bovine_dataset
✅ Model copied to /content/best_model.pth
DATA_DIR -> /content/indian_bovine_dataset/Indian_bovine_breeds
Local model -> /content/best_model.pth


In [ ]:
c

In [ ]:
!kaggle datasets download -d lukex9442/indian-bovine-breeds -p /content -q
!unzip -q /content/indian-bovine-breeds.zip -d /content/indian_bovine_dataset

print("✅ Dataset downloaded and extracted to /content/indian_bovine_dataset")

Dataset URL: https://www.kaggle.com/datasets/lukex9442/indian-bovine-breeds
License(s): CC0-1.0
✅ Dataset downloaded and extracted to /content/indian_bovine_dataset


In [ ]:
# Copy runtime dataset to Drive (run once)
!cp -r /content/indian_bovine_dataset /content/drive/MyDrive/indian_bovine_dataset
# verify
!ls -la /content/drive/MyDrive/ | grep indian_bovine_dataset || true




drwx------ 4 root root     4096 Sep 13 21:54 indian_bovine_dataset


In [ ]:
# Step 5: Data audit and preparation
import os
from PIL import Image, ImageFile
import numpy as np
from collections import defaultdict
import pandas as pd

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ✅ Correct dataset path (point directly to classes)
data_dir = "/content/indian_bovine_dataset/Indian_bovine_breeds"

# Discover classes
classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])

print(f"📂 Found {len(classes)} classes.")
print("✅ Sample class names:", classes[:10])

class_counts = {}
size_stats = defaultdict(list)
corrupt_files = []

for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    files = [f for f in os.listdir(cls_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    class_counts[cls] = len(files)
    for f in files:
        path = os.path.join(cls_dir, f)
        try:
            img = Image.open(path)
            w, h = img.size
            size_stats[cls].append((w, h))
        except Exception:
            corrupt_files.append(path)

# Build summary table
summary = []
for cls in classes:
    sizes = size_stats[cls]
    if sizes:
        ws, hs = zip(*sizes)
        summary.append({
            "class": cls,
            "count": class_counts[cls],
            "w_min": min(ws), "w_max": max(ws), "w_mean": np.mean(ws),
            "h_min": min(hs), "h_max": max(hs), "h_mean": np.mean(hs)
        })
    else:
        summary.append({
            "class": cls,
            "count": class_counts[cls],
            "w_min": None, "w_max": None, "w_mean": None,
            "h_min": None, "h_max": None, "h_mean": None
        })

df_summary = pd.DataFrame(summary).sort_values("count", ascending=True)

print("\n📊 Class distribution (first 10 rows):")
print(df_summary.head(10))
print("\nTotal images:", sum(class_counts.values()))
print("Corrupt files detected:", len(corrupt_files))

if corrupt_files:
    print("⚠️ Corrupt file sample:", corrupt_files[:5])

# Save audit report
audit_csv = os.path.join(data_dir, "data_audit_report.csv")
df_summary.to_csv(audit_csv, index=False)
print(f"\n✅ Audit report saved at: {audit_csv}")


📂 Found 41 classes.
✅ Sample class names: ['Alambadi', 'Amritmahal', 'Ayrshire', 'Banni', 'Bargur', 'Bhadawari', 'Brown_Swiss', 'Dangi', 'Deoni', 'Gir']

📊 Class distribution (first 10 rows):
         class  count  w_min  w_max       w_mean  h_min  h_max      h_mean
20   Kherigarh     36    200   2080   989.916667    147   1368  618.805556
19    Kenkatha     55    150   4272   718.127273    104   2848  502.618182
36       Surti     59    200   1920   852.271186    145   1200  552.525424
39  Umblachery     76    220   4000  1333.236842    147   3000  813.539474
7        Dangi     82    150   5184  1146.804878    150   3456  713.280488
29      Nimari     84    220   5312  1363.928571    147   3456  841.392857
5    Bhadawari     86    211    394   270.593023    122    225  183.488372
26      Nagori     88    196   5184  1366.352273    144   3648  864.840909
28   Nili_Ravi     88    200   2048   663.238636    113   1496  489.590909
16    Kangayam     91    145   5184   870.890110    150   

In [ ]:
import os
import torch  # 👈 ye line missing thi

# ✅ Correct dataset path (go one level deeper)
DATA_DIR = "/content/indian_bovine_dataset/Indian_bovine_breeds"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS = 10

print("Using device:", DEVICE)

# ✅ List only valid class directories
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
print(f"📂 Found {len(classes)} classes.")
print("✅ Sample classes:", classes[:10])


Using device: cuda
📂 Found 41 classes.
✅ Sample classes: ['Alambadi', 'Amritmahal', 'Ayrshire', 'Banni', 'Bargur', 'Bhadawari', 'Brown_Swiss', 'Dangi', 'Deoni', 'Gir']


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader

# --- Transforms ---
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),  # better than fixed resize
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --- Full dataset ---
full_dataset = ImageFolder(DATA_DIR, transform=train_tfms)

# --- Train/Validation split (80/20) ---
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply val transforms to val dataset
val_dataset.dataset.transform = val_tfms

# --- Dataloaders ---
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# --- Check data ---
print(f"Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")
print("Classes:", full_dataset.classes)
print(f"Total classes: {len(full_dataset.classes)}")


Train samples: 4741 | Val samples: 1186
Classes: ['Alambadi', 'Amritmahal', 'Ayrshire', 'Banni', 'Bargur', 'Bhadawari', 'Brown_Swiss', 'Dangi', 'Deoni', 'Gir', 'Guernsey', 'Hallikar', 'Hariana', 'Holstein_Friesian', 'Jaffrabadi', 'Jersey', 'Kangayam', 'Kankrej', 'Kasargod', 'Kenkatha', 'Kherigarh', 'Khillari', 'Krishna_Valley', 'Malnad_gidda', 'Mehsana', 'Murrah', 'Nagori', 'Nagpuri', 'Nili_Ravi', 'Nimari', 'Ongole', 'Pulikulam', 'Rathi', 'Red_Dane', 'Red_Sindhi', 'Sahiwal', 'Surti', 'Tharparkar', 'Toda', 'Umblachery', 'Vechur']
Total classes: 41


In [ ]:
from torchvision import datasets
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split

# Step 8: Dataset setup with stratified split
base_dataset = datasets.ImageFolder(DATA_DIR)  # only to read targets & classes

# Stratified split indices
train_idx, val_idx = train_test_split(
    list(range(len(base_dataset.targets))),
    test_size=0.2,
    stratify=base_dataset.targets,
    random_state=42
)

# Train & validation datasets with correct transforms
train_dataset = Subset(
    datasets.ImageFolder(DATA_DIR, transform=train_tfms),
    train_idx
)
val_dataset = Subset(
    datasets.ImageFolder(DATA_DIR, transform=val_tfms),
    val_idx
)

# DataLoaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True, num_workers=2, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE,
    shuffle=False, num_workers=2, pin_memory=True
)

print(f"Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")
print("Classes:", base_dataset.classes)
NUM_CLASSES = len(base_dataset.classes)


Train samples: 4741 | Val samples: 1186
Classes: ['Alambadi', 'Amritmahal', 'Ayrshire', 'Banni', 'Bargur', 'Bhadawari', 'Brown_Swiss', 'Dangi', 'Deoni', 'Gir', 'Guernsey', 'Hallikar', 'Hariana', 'Holstein_Friesian', 'Jaffrabadi', 'Jersey', 'Kangayam', 'Kankrej', 'Kasargod', 'Kenkatha', 'Kherigarh', 'Khillari', 'Krishna_Valley', 'Malnad_gidda', 'Mehsana', 'Murrah', 'Nagori', 'Nagpuri', 'Nili_Ravi', 'Nimari', 'Ongole', 'Pulikulam', 'Rathi', 'Red_Dane', 'Red_Sindhi', 'Sahiwal', 'Surti', 'Tharparkar', 'Toda', 'Umblachery', 'Vechur']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
from tqdm import tqdm

# Step 9: Model setup
model = timm.create_model("resnet50", pretrained=True, num_classes=NUM_CLASSES)
model = model.to(DEVICE)

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2, factor=0.5)

# Mixed precision scaler (for GPU training)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

# Training & validation functions
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for imgs, labels in tqdm(loader, leave=False):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss/total, correct/total

def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss/total, correct/total

# Training loop with checkpoint saving
best_acc = 0.0
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"✅ Best model saved at epoch {epoch+1} with Val Acc: {val_acc:.4f}")


/tmp/ipython-input-3693738621.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
  0%|          | 0/149 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/tmp/ipython-input-3693738621.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
 11%|█         | 16/149 [00:12<01:06,  1.99it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10 | Train Loss: 3.5214, Acc: 0.0932 | Val Loss: 3.2918, Acc: 0.1214
✅ Best model saved at epoch 1 with Val Acc: 0.1214


 28%|██▊       | 41/149 [00:30<01:39,  1.09it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 2/10 | Train Loss: 3.0001, Acc: 0.2073 | Val Loss: 2.7533, Acc: 0.2766
✅ Best model saved at epoch 2 with Val Acc: 0.2766


 30%|██▉       | 44/149 [00:28<00:47,  2.21it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 8/10 | Train Loss: 1.2606, Acc: 0.6220 | Val Loss: 1.5573, Acc: 0.5245
✅ Best model saved at epoch 8 with Val Acc: 0.5245


 42%|████▏     | 63/149 [00:45<01:01,  1.39it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 43%|████▎     | 64/149 [00:47<01:19,  1.07it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 9/10 | Train Loss: 1.1574, Acc: 0.6558 | Val Loss: 1.4961, Acc: 0.5388
✅ Best model saved at epoch 9 with Val Acc: 0.5388


 28%|██▊       | 42/149 [00:32<01:27,  1.22it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 57%|█████▋    | 85/149 [01:02<00:31,  2.01it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 10/10 | Train Loss: 1.0680, Acc: 0.6834 | Val Loss: 1.4417, Acc: 0.5455
✅ Best model saved at epoch 10 with Val Acc: 0.5455


In [ ]:
# ======================================
# Continue Training: Extra 10 epochs
# ======================================

# Reload the best saved model (so far)
model = timm.create_model("resnet50", pretrained=False, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model = model.to(DEVICE)

# Redefine optimizer & scheduler (same as before)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2, factor=0.5)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

# Continue training for +10 epochs
extra_epochs = 10
start_epoch = 10  # because you already trained 10
best_acc = 0.0    # re-init, or reload from logs if you tracked

for epoch in range(start_epoch, start_epoch + extra_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{start_epoch+extra_epochs} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"✅ Best model saved at epoch {epoch+1} with Val Acc: {val_acc:.4f}")


/tmp/ipython-input-2772822426.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
  0%|          | 0/149 [00:00<?, ?it/s]/tmp/ipython-input-3693738621.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
 49%|████▉     | 73/149 [00:54<00:52,  1.44it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 52%|█████▏    | 77/149 [00:56<00:40,  1.79it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 11/20 | Train Loss: 0.9588, Acc: 0.7129 | Val Loss: 1.4069, Acc: 0.5590
✅ Best model saved at epoch 11 with Val Acc: 0.5590


 36%|███▌      | 53/149 [00:37<00:42,  2.28it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 66%|██████▌   | 98/149 [01:09<00:30,  1.65it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 12/20 | Train Loss: 0.8606, Acc: 0.7374 | Val Loss: 1.3935, Acc: 0.5573


  3%|▎         | 4/149 [00:05<02:31,  1.05s/it]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 35%|███▍      | 52/149 [00:40<01:02,  1.56it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 18/20 | Train Loss: 0.5051, Acc: 0.8557 | Val Loss: 1.4173, Acc: 0.5877


  9%|▊         | 13/149 [00:08<01:23,  1.64it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 60%|██████    | 90/149 [01:05<00:47,  1.25it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 19/20 | Train Loss: 0.4654, Acc: 0.8663 | Val Loss: 1.4262, Acc: 0.5700


 41%|████      | 61/149 [00:45<01:02,  1.40it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 83%|████████▎ | 123/149 [01:29<00:23,  1.10it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
                                               

Epoch 20/20 | Train Loss: 0.4301, Acc: 0.8783 | Val Loss: 1.4045, Acc: 0.5835


In [ ]:
# ======================================
# Fine-tuning last layers after plateau
# ======================================

# 1. Reload best model checkpoint
model = timm.create_model("resnet50", pretrained=False, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model = model.to(DEVICE)

# 2. Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# 3. Unfreeze last block (layer4) + classifier head
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name or "head" in name:
        param.requires_grad = True

# 4. Redefine optimizer (smaller LR for fine-tuning)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # with label smoothing
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2, factor=0.5)

# Mixed precision scaler
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

# 5. Fine-tuning loop
extra_epochs = 10   # try 5–10
start_epoch = 20    # because you already trained 20 total (10 + 10)

best_acc = 0.0
for epoch in range(start_epoch, start_epoch + extra_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    scheduler.step(val_loss)

    print(f"[Fine-tune] Epoch {epoch+1}/{start_epoch+extra_epochs} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"✅ Fine-tuned best model saved at epoch {epoch+1} with Val Acc: {val_acc:.4f}")


/tmp/ipython-input-4107465363.py:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
  0%|          | 0/149 [00:00<?, ?it/s]/tmp/ipython-input-3693738621.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
  4%|▍         | 6/149 [00:04<01:20,  1.79it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[Fine-tune] Epoch 21/30 | Train Loss: 1.3708, Acc: 0.8374 | Val Loss: 2.0077, Acc: 0.5826
✅ Fine-tuned best model saved at epoch 21 with Val Acc: 0.5826


 17%|█▋        | 25/149 [00:18<01:28,  1.40it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 77%|███████▋  | 115/149 [01:19<00:15,  2.23it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[Fine-tune] Epoch 22/30 | Train Loss: 1.3559, Acc: 0.8336 | Val Loss: 2.0111, Acc: 0.5801


 70%|██████▉   | 104/149 [01:10<00:30,  1.45it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 83%|████████▎ | 124/149 [01:24<00:14,  1.69it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

[Fine-tune] Epoch 24/30 | Train Loss: 1.3132, Acc: 0.8498 | Val Loss: 1.9628, Acc: 0.5894
✅ Fine-tuned best model saved at epoch 24 with Val Acc: 0.5894


 44%|████▎     | 65/149 [00:47<01:12,  1.16it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 53%|█████▎    | 79/149 [00:54<00:35,  1.99it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[Fine-tune] Epoch 25/30 | Train Loss: 1.3057, Acc: 0.8401 | Val Loss: 1.9516, Acc: 0.5877


 14%|█▍        | 21/149 [00:17<01:26,  1.48it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 36%|███▌      | 53/149 [00:39<01:32,  1.03it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from PIL import Image
import torch
from torchvision import transforms

# ImageNet normalization (same as training)
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# --- Save class names once (from your dataset) ---
CLASS_NAMES = base_dataset.classes   # Step 8 ke baad base_dataset defined hota hai

def predict(img_path):
    img = Image.open(img_path).convert("RGB")
    tensor = val_tfms(img).unsqueeze(0).to(DEVICE)   # shape (1,3,224,224)

    model.eval()
    with torch.no_grad():
        outputs = model(tensor)
        _, pred = torch.max(outputs, 1)

    return CLASS_NAMES[pred.item()]  # Use saved class list


In [ ]:
!mkdir -p /content/drive/MyDrive/buffalo_models
!cp best_model.pth /content/drive/MyDrive/buffalo_models/best_model.pth


In [ ]:
test_img = "/content/BS.jpeg"
print("Predicted breed:", predict(test_img))


NameError: name 'predict' is not defined